# An Introduction to Reinforcement Learning

## [Tom Bewley](https://tombewley.com/) & [Scott Jeen](https://enjeeneer.io/)

### 24/02/2022


# 1 | Markov Decision Processes: A Model of Sequential Decision Making

## 1.1. MDP (semi-)Formalism 

In reinforcement learning (RL), an *agent* takes *actions* in an *environment* to change its state, with the goal of maximising the expected sum of future *rewards*. We formalise this interaction as an agent-environment loop, mathematically described as a Markov Decision Process (MDP). 

<img src='https://github.com/enjeeneer/sutton_and_barto/blob/main/images/chapter3_1.png?raw=true' width='700'>

MDPs break the I.I.D. data assumption of supervised and unsupervised learning; the agent *causally influences* the data it sees through its choice of actions. However, one assumption we do make is the *Markov property*, which says that the state representation captures *all relevent information* from the past. Formally, state transitions depend only on the most recent state and action,
$$
\mathbb{P}[S_{t+1} | S_t,A_t] = \mathbb{P}[S_{t+1} | S_1,A_1 \ldots\, S_t,A_t],
$$
and rewards depend only on the most recent transition,
$$
\mathbb{P}[R_{t+1} | S_t,A_t,S_{t+1}] = \mathbb{P}[R_{t+1} | S_1,A_1 \ldots\, S_t,A_t,S_{t+1}].
$$
- Note: different sources use different notation here, but this is the most general.

In some MDPs, a subset of states are designated as *terminal* (or *absorbing*). The agent-environment interaction loop ceases once a terminal state is reached.

The goal of an RL agent is to pick actions that maximise the discounted cumulative sum of future rewards, also known as the *return* $G_t$:
$$
G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \ldots + \gamma^{T-t-1}R_{T},
$$
where $\gamma\in[0,1]$ is a discount factor and $T$ is the time of termination (may be $\infty$).

To do so, it needs the ability to forecast the reward-getting effect of taking each action $A$ in each state $S$, potentially many timesteps into the future. This *temporal credit assignment* problem is one of the key factors that makes RL so challenging.

## 1.2 MDP Example

Here's a simple MDP (courtesy of David Silver @ DeepMind/UCL), which we'll be using throughout this course.
- White circle: non-terminal state
- White square: terminal state
- Black circle: action
- <span style="color:green">Green:</span> reward (depends only on $S_{t+1}$ here)
- <span style="color:blue">Blue:</span> state transition probability
- <span style="color:red">Red:</span> action probability for an exemplar policy
- Note: edges with probability $1$ are unlabelled

<img src='https://github.com/tombewley/one-hour-rl/blob/main/images/student-mdp.svg?raw=true' width='700'>

## 1.3 Open AI Gym

[Open AI Gym](https://gym.openai.com/) provides a unified framework for testing and comparing RL algorithms, and offers a suite of MDPs that researchers can use to benchmark their work. It's important to be familiar with the conventions of Gym, because almost all modern RL code is built to work with it. Gym environment classes have two key methods:

- `mdp.reset()`: resets the MDP to an initial state $S_0$ according to an initialisation distribution.
- `mdp.step(action)` : takes an action $A_t$, combines with the current environment state $S_t$, produces the next state $S_{t+1}$ and delivers the agent a scalar reward $R_{t+1}$.

A Gym-compatible class for the student MDP shown above can be found in `mdp.py` in this repository. Let's import it now and explore what it can do!

In [1]:
from mdp import StudentMDP
mdp = StudentMDP()

Firstly, we'll have a look at the initialisation probabilities and the behaviour of `mdp.reset()`.

In [3]:
print(mdp.initial_probs())
mdp.reset()
print(mdp.state)

{'Class 1': 1.0, 'Class 2': 0.0, 'Class 3': 0.0, 'Facebook': 0.0, 'Pub': 0.0, 'Pass': 0.0, 'Asleep': 0.0}
Class 1


Next, let's check which actions are available in this initial state, and the action-dependent transition probabilities.
- Reminder: the Markov property dictates that transition probabilities depend *only* on the current state and action.

In [4]:
print(mdp.action_space(mdp.state))
print(mdp.transition_probs(mdp.state, "Study"))

{'Study', 'Go on Facebook'}
{'Class 2': 1.0}


Calling `mdp.step(action)` samples and returns the next state $S_{t+1}$, alongside a scalar reward $R_{t+1}$.

Let's try calling this method repeatedly. What's happening here?

In [6]:
state, reward, _, _ = mdp.step("Study") 
print(state, reward)

Class 3 -2.0


Transitions out of the `Pub` state are *stochastic*; they go to one of the three classes with specified probabilities.

In [7]:
mdp.state = "Pub"
print(mdp.action_space(mdp.state))
print(mdp.transition_probs(mdp.state, "Have a pint"))

{'Have a pint'}
{'Class 1': 0.2, 'Class 2': 0.4, 'Class 3': 0.4}


In this state, the behaviour of `mdp.step(action)` is non-deterministic, even for a constant action.

In [8]:
mdp.state = "Pub" # Note that we're resetting the state to Pub each time
state, reward, _, _ = mdp.step("Have a pint")
print(state, reward)

Class 1 -2.0


This MDP has just one terminal state.

In [9]:
print(mdp.terminal_states())

{'Asleep'}


`mdp.step(action)` also returns a binary `done` flag, which is set to `True` if $S_{t+1}$ is a terminal state.

In [10]:
mdp.state = "Class 2" 
state, reward, done, _ = mdp.step("Fall asleep")
print(state, reward, done)

mdp.state = "Pass" 
state, reward, done, _ = mdp.step("Fall asleep")
print(state, reward, done)

Asleep 0.0 True
Asleep 0.0 True


Now let's bring an agent into the mix, and give it the exemplar policy shown in the diagram above.

In [11]:
from agent import Agent
agent = Agent(mdp) 
agent.policy = {
    "Class 1":  {"Study": 0.5, "Go on Facebook": 0.5},
    "Class 2":  {"Study": 0.8, "Fall asleep": 0.2},
    "Class 3":  {"Study": 0.6, "Go to the pub": 0.4},
    "Facebook": {"Keep scrolling": 0.9, "Close Facebook": 0.1},
    "Pub":      {"Have a pint": 1.},
    "Pass":     {"Fall asleep": 1.},
    "Asleep":   {"Stay asleep": 1.}
}

We can query the policy in a similar way to the MDP's properties, and observe its stochastic behaviour.

In [12]:
print(agent.policy["Class 1"])
print([agent.act("Class 1") for _ in range(20)])

{'Study': 0.5, 'Go on Facebook': 0.5}
['Go on Facebook', 'Study', 'Go on Facebook', 'Study', 'Go on Facebook', 'Go on Facebook', 'Study', 'Go on Facebook', 'Go on Facebook', 'Go on Facebook', 'Study', 'Go on Facebook', 'Study', 'Go on Facebook', 'Study', 'Study', 'Study', 'Go on Facebook', 'Study', 'Go on Facebook']


Bringing it all together

In [14]:
mdp.verbose = True
state = mdp.reset()
done = False
while not done:
    state, reward, done, info = mdp.step(agent.act(state))

=========================== EPISODE   4 ===========================
| Time  | State    | Action         | Reward | Next state | Done  |
|-------|----------|----------------|--------|------------|-------|
| 0     | Class 1  | Go on Facebook | -1.0   | Facebook   | False |
| 1     | Facebook | Keep scrolling | -1.0   | Facebook   | False |
| 2     | Facebook | Keep scrolling | -1.0   | Facebook   | False |
| 3     | Facebook | Keep scrolling | -1.0   | Facebook   | False |
| 4     | Facebook | Keep scrolling | -1.0   | Facebook   | False |
| 5     | Facebook | Keep scrolling | -1.0   | Facebook   | False |
| 6     | Facebook | Keep scrolling | -1.0   | Facebook   | False |
| 7     | Facebook | Keep scrolling | -1.0   | Facebook   | False |
| 8     | Facebook | Keep scrolling | -1.0   | Facebook   | False |
| 9     | Facebook | Keep scrolling | -1.0   | Facebook   | False |
| 10    | Facebook | Keep scrolling | -1.0   | Facebook   | False |
| 11    | Facebook | Keep scrolling | -1.0   | F

How "good" is this policy? To answer this, we need to calculate its expected return.

# 2 | Policy Evaluation: The Temporal Difference Method

In RL, the agent selects actions from its policy $\pi$, which is a mapping from states to actions $\mathcal{S} \rightarrow \mathcal{A}$. In the previous example the policy was formalised as a series of action probabilities given a state. An important aspect of RL, is quantifying how good a policy is as achieving the goal we'd like to achieve. We do this using *Value Functions*, the most common being the state-action value function $Q(s_t, a_t)$.

$Q$ estimates the expected future return from a state $s_t$ if action $a_t$ is chosen, and the policy $\pi$ is followed at all subsequent states. By sampling episodes in our MDP using the current policy we can collect rewards and update our Q-function accordingly. The algorithm we use to evaluate policies is called policy evaluation, and it uses the Bellman back-up which has two hyperparameters $\gamma$ and $\alpha$. $\gamma$ is the discount factor that

Import same MDP and policy as last time

In [20]:
from mdp import StudentMDP
from agent import Agent
mdp = StudentMDP(verbose=True)
agent = Agent(mdp) 
agent.policy = {
    "Class 1":  {"Study": 0.5, "Go on Facebook": 0.5},
    "Class 2":  {"Study": 0.8, "Fall asleep": 0.2},
    "Class 3":  {"Study": 0.6, "Go to the pub": 0.4},
    "Facebook": {"Keep scrolling": 0.9, "Close Facebook": 0.1},
    "Pub":      {"Have a pint": 1.},
    "Pass":     {"Fall asleep": 1.},
    "Asleep":   {"Stay asleep": 1.}
}

In [21]:
agent.Q

{'Class 1': {'Study': 0.0, 'Go on Facebook': 0.0},
 'Class 2': {'Study': 0.0, 'Fall asleep': 0.0},
 'Class 3': {'Study': 0.0, 'Go to the pub': 0.0},
 'Facebook': {'Keep scrolling': 0.0, 'Close Facebook': 0.0},
 'Pub': {'Have a pint': 0.0},
 'Pass': {'Fall asleep': 0.0},
 'Asleep': {}}

In [22]:
GAMMA = 0.9
ALPHA = 0.01

def bellman_backup(agent, state, action, reward, next_state, done):

    Q_next = 0. if done else agent.Q[next_state][agent.act(next_state)]

    agent.Q[state][action] += ALPHA * ( reward + GAMMA * Q_next - agent.Q[state][action])

In [59]:
for _ in range(1):
    state = mdp.reset()
    done = False
    while not done:
        action = agent.act(state)
        next_state, reward, done, _ = mdp.step(action)

        print(agent.Q[state][action])
        bellman_backup(agent, state, action, reward, next_state, done)
        print(agent.Q[state][action])

        state = next_state

=========================== EPISODE  37 ===========================
| Time  | State    | Action         | Reward | Next state | Done  |
|-------|----------|----------------|--------|------------|-------|
| 0     | Class 1  | Study          | -2.0   | Class 2    | False |
-0.7314255931549986
-0.7441113372234486
| 1     | Class 2  | Study          | -2.0   | Class 3    | False |
-0.39231268945788256
-0.4069362679472403
| 2     | Class 3  | Study          | 10.0   | Pass       | False |
2.3765728565289628
2.452807127963673
| 3     | Pass     | Fall asleep    |  0.0   | Asleep     | True  |
0.0
0.0


Try with $\gamma=0$

# 3 | Policy Improvement: The Q-learning Algorithm

Having evaluated our policy $\pi$, how can we go about obtaining a better one? This question is the heart of *policy improvement*, perhaps the fundamental concept of RL. Recall, when we performed policy evaluation we obtained the value of taking every action in every state. Thus, we can perform policy improvement readily by picking our current best estimate of the optimal action from each state -- so-called *greedy* action selection. Once we've obtained a new policy, we can evaluate it as before. Continually iterating between policy evaluation and policy improvement in this way, we are guarenteed to reach the optimal policy $\pi^*$ according to the policy improvement theorem.

In [60]:
from mdp import StudentMDP
mdp = StudentMDP(verbose=True)

In [61]:
from agent import QLearningAgent
agent = QLearningAgent(mdp, epsilon=1.0, alpha=0.2, gamma=0.9)

In [62]:
NUM_EPS = 50
mdp.ep = 0
while mdp.ep < NUM_EPS:
    state = mdp.reset()
    done = False
    while not done:
        action = agent.act(state)
        next_state, reward, done, info = mdp.step(action)
        agent.learn(state, action, reward, next_state, done)
        state = next_state

    print("Value function:")
    print(agent.Q)
    print("Policy:")
    print(agent.policy)
    print("Epsilon:", agent.epsilon)
    
    agent.epsilon = max(agent.epsilon - 1 / (NUM_EPS-1), 0)

=========================== EPISODE   1 ===========================
| Time  | State    | Action         | Reward | Next state | Done  |
|-------|----------|----------------|--------|------------|-------|
| 0     | Class 1  | Study          | -2.0   | Class 2    | False |
| 1     | Class 2  | Study          | -2.0   | Class 3    | False |
| 2     | Class 3  | Go to the pub  |  1.0   | Pub        | False |
| 3     | Pub      | Have a pint    | -2.0   | Class 3    | False |
| 4     | Class 3  | Study          | 10.0   | Pass       | False |
| 5     | Pass     | Fall asleep    |  0.0   | Asleep     | True  |
Value function:
{'Class 1': {'Study': -0.4, 'Go on Facebook': 0.0}, 'Class 2': {'Study': -0.4, 'Fall asleep': 0.0}, 'Class 3': {'Study': 2.0, 'Go to the pub': 0.2}, 'Facebook': {'Keep scrolling': 0.0, 'Close Facebook': 0.0}, 'Pub': {'Have a pint': -0.36400000000000005}, 'Pass': {'Fall asleep': 0.0}, 'Asleep': {}}
Policy:
{'Class 1': {'Study': 0.5, 'Go on Facebook': 0.5}, 'Class 2': {'S

Try with $\gamma=0$ and different $\epsilon$ values

# 4 | Deep RL


So far, we've tabularised the state-action space. Whilst useful for explaining the fundamental concepts that underpin RL, the real world state-action spaces are generally continuous and thus impossible to tabularise. To combat this, function approximators are used instead. In the past these included x, but more recently, deep neural networks have been used giving rise to the field of Deep Reinforcement Learning.

The seminal Deep RL algorithm is Deep Q Learning which uses neural networks to represent the $Q$ function. The network takes the current obervation $o_t$ as input and predicts the value of each action. The agent's policy is $\epsilon$-greedy as before i.e. it takes the value-maximising action with probability $1 - \epsilon$. Deep Q learning 

Below, we run 500 episodes of the canonical Cartpole task using Deep Q learning. The agent's goal is to balance the pole in the upright position for as long as possible starting from an initially random position.

In [63]:
import gym
from dqn_agent import Agent
import numpy as np

In [ ]:
env = gym.make('CartPole-v1')
agent = Agent(gamma=0.99, epsilon=0.9, lr=0.0001, n_actions=env.action_space.n, input_dims=[env.observation_space.shape[0]],
              mem_size=50000, batch_size=128,  eps_dec=1e-3, eps_min=0.05, replace=1000,
              env_name='cartpole', chkpt_dir='tmp/dqn')

best_score = -np.inf
episodes = 500
scores, avg_score, eps_history = [], [], []

for i in range(episodes):
    score = 0
    done = False
    observation = env.reset()
    env.render()
    while not done:
        action = agent.choose_action(observation)
        observation_, reward, done, info = env.step(action)
        score += reward
        agent.store_transition(observation, action, reward, observation_, done)
        agent.learn()
        observation = observation_
        env.render()
    
    scores.append(score)
    eps_history.append(agent.epsilon)
    
    avg_score = np.mean(scores[-100:])
    
    print('episode', i, 'score %.2f' % score, 'average score %.2f' % avg_score)

episode 0 score 17.00 average score 17.00
episode 1 score 28.00 average score 22.50
episode 2 score 16.00 average score 20.33
episode 3 score 14.00 average score 18.75
episode 4 score 10.00 average score 17.00
episode 5 score 45.00 average score 21.67
episode 6 score 16.00 average score 20.86
episode 7 score 17.00 average score 20.38
episode 8 score 31.00 average score 21.56
episode 9 score 26.00 average score 22.00
episode 10 score 13.00 average score 21.18
episode 11 score 14.00 average score 20.58
episode 12 score 30.00 average score 21.31
episode 13 score 62.00 average score 24.21
episode 14 score 13.00 average score 23.47
episode 15 score 26.00 average score 23.62
episode 16 score 11.00 average score 22.88
episode 17 score 28.00 average score 23.17
episode 18 score 31.00 average score 23.58
episode 19 score 19.00 average score 23.35
episode 20 score 18.00 average score 23.10
episode 21 score 23.00 average score 23.09
episode 22 score 18.00 average score 22.87
episode 23 score 37.0

In [ ]:
import matplotlib.pyplot as plt
figure, ax1 = plt.subplots(1,1, figsize=(12,7.5))
ax1.plot(np.arange(episodes), avg_score, label='rolling avg reward', color='blue')
ax1.set_xlabel('episodes')
ax1.set_ylabel('score', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

ax2 = ax1.twinx()
ax2.plot(np.arange(episodes), eps_history, label='epsilon', color='red')
ax2.set_ylabel('epsilon', color='red')
ax2.tick_params(axis='y', labelcolor='red')
plt.show()

# 5 | What Did We Miss Out?

- Dynamic programming (when transition probabilities are known)
- Monte Carlo
- Exploration strategies
- Continuous actions
- Policy gradient, actor-critic
- Model-based
- Partial observability

What next? RL interest group?